In [1]:
import json
import math
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import warnings

In [2]:
BASE_HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}

In [ ]:
# total_page = 206

In [ ]:
# PROXIES = {
#     "http://178.155.104.84:59071",
#     "http://178.155.104.84:59071"
# }
# PROXIES = {"http://11.456.448.110:8080"}

In [27]:
def scrape_one_page(url):
    parent_data = []
    page = requests.get(url, headers=BASE_HEADERS)
    soup = BeautifulSoup(page.content, 'html.parser')
    all_items = soup.find_all('div', {'class':'BasePropertyCard_propertyCardWrap__J0xUj'})
    
    for item in all_items:
        status = item.find('div', {'class':'base__StyledType-rui__sc-108xfm0-0 kpUjhd message'}).text
        
        try:
            price = item.find(
                'div', {'class':'Pricestyles__StyledPrice-rui__btk3ge-0 bvgLFe card-price" data-testid="card-price"'}
            ).text
        except: price = np.nan
        
        try:
            bed_parent = item.find(
                'li', "PropertyBedMetastyles__StyledPropertyBedMeta-rui__a4nnof-0 cHVLag"
            )
            bed = bed_parent.find('span', {'data-testid':'meta-value'}).text
        except: bed = np.nan
    
        try:
            bath_parent = item.find(
                "li", 'PropertyBathMetastyles__StyledPropertyBathMeta-rui__sc-67m6bo-0 bSPXLm',
            )
            bath = bath_parent.find("span", {"data-testid": "meta-value"}).text
        except: bath = np.nan
            
        try:
            sqft_parent = item.find(
                "li", 'PropertySqftMetastyles__StyledPropertySqftMeta-rui__sc-1gdau7i-0 fnhaOV',
            )
            sqft = sqft_parent.find("span", {"data-testid": "meta-value"}).text
        except: sqft = np.nan
            
        try:
            size_parent = item.find(
                "li", "PropertyLotSizeMetastyles__StyledPropertyLotSizeMeta-rui__sc-1cz4zco-0 cNMyen",
            )
            size = size_parent.find("span", {"data-testid": "meta-value"}).text
        except: size = np.nan
    
        try:
            address1 = item.find("div", {"data-testid": "card-address-1"}).text
        except: address1 = np.nan
            
        try:
            address2 = item.find("div", {"data-testid": "card-address-2"}).text
        except: address2 = np.nan
    
        try:
            product_link = item.find("a", "LinkComponent_anchor__2uAhr")["href"]
            product_link = "https://www.realtor.com" + product_link
        except: product_link = np.nan
            
        data = [
            product_link,
            status,
            price,
            bed,
            bath,
            sqft,
            size,
            address1,
            address2,
        ]
        parent_data.append(data)


In [31]:
for page in range (1, 206):
    df = pd.DataFrame()
    url = 'https://www.realtor.com/realestateandhomes-search/New-York_NY/pg-{}'.format(page)
    scrape_one_page(url)
    cols = [
        "Product URL",
        "Status",
        "Price",
        "Bed",
        "Bath",
        "Sqft",
        "Acre Lot",
        "Address 1",
        "Address 2",
    ]
    df = pd.DataFrame(columns=cols, data=parent_data)
    df.to_csv('./realtor_1.csv', sep=';', encoding='UTF-8', index=False, mode='a')
    time.sleep(random.randint(1, 50))